In [1]:
#@title Imports and load google drive, navigate to BPD folder
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
import multiprocessing as mp


In [6]:
data = pd.read_csv('new_d.csv')
data = data[['ID', 'Condition', 'Session', 'BPD#']]
new_data = []
subject = 0
for ids in data.ID.unique():
    if len(data.loc[data['ID'] == ids, 'Session'].unique()) < 2:
        continue
    else:
        temp_data = data.loc[data['ID'] == ids]
        temp_data['subject'] = subject
        new_data.append(temp_data)
        subject = subject + 1
new_data = pd.concat(new_data).drop('ID', axis = 1)

In [61]:
traces = pd.read_csv('model_traces/10k40k5p6thin_indsubj_8chain_traces.csv').drop('Unnamed: 0', axis = 1)
traces['trace#'] = np.tile(np.arange(1000), 148)
traces_melt = traces.melt(id_vars=['subject', 'trace#'], var_name='parameters', value_name='param_vals')
traces_melt['Session'] = 'Stressed'
traces_melt.loc[traces_melt['parameters'].str.contains('Neutral'), 'Session'] = 'Neutral'
for sub in new_data['subject'].unique():
    traces_melt.loc[traces_melt['subject'] == sub, 'BPD'] = new_data.loc[new_data['subject'] == sub, 'BPD#'].unique()[0]
    traces_melt.loc[traces_melt['subject'] == sub, 'Condition'] = new_data.loc[new_data['subject'] == sub, 'Condition'].unique()[0]

traces_melt.loc[traces_melt['Condition'] == 1, 'Condition'] = 'Non Social'
traces_melt.loc[traces_melt['Condition'] == 2, 'Condition'] = 'Social'
traces_melt['mean_centered_BPD'] = traces_melt['BPD'] - np.mean(traces_melt['BPD'].values)


In [62]:
def fit_lm(trace_no):
    param = 'v'
    temp_df = traces_melt.loc[((traces_melt['trace#'] == trace_no) & (traces_melt['parameters'].str.contains(param)))]
    res = smf.ols(formula='param_vals ~  mean_centered_BPD*Session*C(Condition)', data=temp_df).fit()
    if trace_no%100 == 0:
        print(trace_no)
    return res.params

,subject,trace#,parameters,parameter values,Session,BPD,Condition
0,0,0,v(Neutral),-0.234472,Neutral,5.0,Non Social
1,0,1,v(Neutral),-0.180591,Neutral,5.0,Non Social
2,0,2,v(Neutral),0.099004,Neutral,5.0,Non Social
3,0,3,v(Neutral),-0.106410,Neutral,5.0,Non Social
4,0,4,v(Neutral),-0.060558,Neutral,5.0,Non Social
...,...,...,...,...,...,...,...
887995,147,995,pos_alpha(Stressed),-0.472369,Stressed,4.0,Social
887996,147,996,pos_alpha(Stressed),0.497567,Stressed,4.0,Social
887997,147,997,pos_alpha(Stressed),1.219175,Stressed,4.0,Social
887998,147,998,pos_alpha(Stressed),-4.525496,Stressed,4.0,Social


In [ ]:
# def fit_linear(trace_no, param):
y = traces.loc[traces['trace#'] == 0, 'v(Neutral)']

In [4]:
df_neutral = pd.read_csv('df_neutral.csv').drop('Unnamed: 0', axis = 1)
df_stressed = pd.read_csv('df_stressed.csv').drop('Unnamed: 0', axis = 1)


In [5]:
df_neutral

,subj_idx,response,feedback,cond,Session,trial,rt,q_init,split_by,BPD,Condition
0,0,1,1.0,AB,Neutral,0.0,0.5,0.5,0.0,No BPD,1
1,0,1,1.0,EF,Neutral,0.0,0.5,0.5,2.0,No BPD,1
2,0,0,0.0,CD,Neutral,0.0,0.5,0.5,1.0,No BPD,1
3,0,0,0.0,CD,Neutral,1.0,0.5,0.5,1.0,No BPD,1
4,0,1,1.0,AB,Neutral,1.0,0.5,0.5,0.0,No BPD,1
...,...,...,...,...,...,...,...,...,...,...,...
30595,147,1,1.0,CD,Neutral,18.0,0.5,0.5,1.0,No BPD,2
30596,147,1,1.0,AB,Neutral,18.0,0.5,0.5,0.0,No BPD,2
30597,147,1,1.0,AB,Neutral,19.0,0.5,0.5,0.0,No BPD,2
30598,147,1,1.0,CD,Neutral,19.0,0.5,0.5,1.0,No BPD,2
